In [17]:
#cleaning up mmia transcripts 
# 3/12/24 Emily Doherty
import pandas as pd
import os
from datetime import datetime, timedelta

####NOTES
#Need MMIA recordings for 105 + 113

In [18]:
def convert_to_time_format(timestamp_str):
    try:
        # Try parsing timestamp with milliseconds and 'Z' suffix
        timestamp_obj = datetime.strptime(timestamp_str, '%Y-%m-%dT%H:%M:%S.%fZ')
    except ValueError:
        try:
            # Try parsing timestamp without milliseconds and with 'Z' suffix
            timestamp_obj = datetime.strptime(timestamp_str, '%Y-%m-%dT%H:%M:%SZ')
        except ValueError:
            try:
                # Try parsing timestamp without milliseconds and without 'Z' suffix
                timestamp_obj = datetime.strptime(timestamp_str, '%Y-%m-%d %H:%M:%S.%f')
            except ValueError:
                try:
                    # Try parsing timestamp without milliseconds and without 'Z' suffix
                    timestamp_obj = datetime.strptime(timestamp_str, '%Y-%m-%d %H:%M:%S')
                except ValueError:
                    raise ValueError("Timestamp format not supported")
    timestamp_obj -= timedelta(hours=7) ###Because mmia server is ahead 5 hours?
    # Format the datetime object to display only the time in HH:MM format
    time_format = timestamp_obj.strftime('%-I:%M %p')  # '%-I' removes leading zero for hour
    
    return time_format

## Import timestamps of Knowledge Sharing and Brainstorming


In [19]:
#import recordingID info
mmiaID=pd.read_excel('/Users/emilydoherty/Desktop/JIA/JIA_recordingIDs.xlsx')

#import KS/BS timestamps
file_path='/Users/emilydoherty/Downloads/JIA_Transcripts_Tracker start_stop times - Tracker.csv'
tracker=pd.read_csv(file_path)
tracker.dropna(axis=1, how='all', inplace=True)
tracker = tracker.drop(tracker.index[0])
tracker = tracker.iloc[:, :-5]
tracker.drop(tracker.columns[1:5], axis=1, inplace=True)
tracker = tracker.rename(columns={tracker.columns[0]: 'Session_ID'})
tracker = tracker.rename(columns={tracker.columns[1]: 'KS_START'})
tracker = tracker.rename(columns={tracker.columns[2]: 'KS_END'})
tracker = tracker.rename(columns={tracker.columns[3]: 'BS_START'})
tracker = tracker.rename(columns={tracker.columns[4]: 'BS_END'})
tracker['Session_ID']=tracker['Session_ID'].astype('int64')

#merge the two
merged_df = pd.merge(mmiaID, tracker, on='Session_ID', how='inner')


## Import RAW MMIA transcripts (first run pull_data_to_csv.py)

In [21]:
#loop thru transcript files 
csv_directory='/Users/emilydoherty/Desktop/JIA/raw_mmia'
output_csv_directory = '/Users/emilydoherty/Desktop/JIA/test'

os.makedirs(output_csv_directory, exist_ok=True)
for filename in os.listdir(csv_directory):
    if filename.endswith(".csv"):
        df = pd.read_csv(os.path.join(csv_directory, filename))
        columns_to_drop = [
            'chunk_link',
            'asr_mode',
            'sessionId',
            'class_id',
            'source',
            'recording_start_date',
            'chunk_start_date',
            'amr',
            'CoBi_Model']
        #not sure what utterance_id is?
        df.drop(columns_to_drop, axis=1, inplace=True)
        # df.dropna(subset=['text'], inplace=True) #dont want to drop rows w/o text bc jia agent doesnt have text populated
        df['new_timestamp'] = df['timestamp'].apply(convert_to_time_format)
        recording_id = df['recordingId'].iloc[0]  # Get recording ID from the CSV file
        
        # Filter the rows based on KS_Start and BS_End for the specific recording ID
        ks_start = merged_df.loc[merged_df['Recording_ID'] == recording_id, 'KS_START'].iloc[0]
        bs_end = merged_df.loc[merged_df['Recording_ID'] == recording_id, 'BS_END'].iloc[0]
        df = df[(df['new_timestamp'] >= ks_start) & (df['new_timestamp'] <= bs_end)]

        # Save the filtered DataFrame back to the CSV file
        output_filename = os.path.join(output_csv_directory, filename)
        df.to_csv(output_filename, index=False)